# **CREACION DE LA BASE DE DATOS DEL PROYECTO**

In [75]:
import pandas as pd
import aiohttp
import asyncio

# import patch to allow an additional run entry point
# for asynchronous functions
import nest_asyncio
nest_asyncio.apply()

from aiohttp import ClientSession

In [76]:
API_KEY = None # se eliminó la API_KEY antes de publicar
URL = "https://api.rawg.io/api/games"

## Base de datos original de Kaggle

La base de datos original que proviene de Kaggle posee las siguientes columnas:

- Rank: Esta columna no se utilizará ya que es solo un orden del total de ventas
- Name: Nombre del videojuego
- Franchise: Franquicia a la que pertenece el videojuego
- Platform: Plataforma a la que pertenece el videojuego, es posible que el videojuego este presente en mas de una plataforma como entrada separada
- Year: Año de lanzamiento del videojuego
- Genre: Genero al que pertenece el videojuego
- Publisher: Empresa responsable de la publicación del videojuego
- NA_Sales: Ventas en norteamérica (en millones de unidades)
- EU_Sales: Ventas en europa (en millones de unidades)
- JP_Sales: Ventas en japón (en millones de unidades)
- Other_Sales: Ventas en el resto del mundo (en millones de unidades)
- Global_Sales: Ventas globales que corresponden a la suma de las tres columnas anteriores

La base de datos ha sido descargada desde https://www.kaggle.com/datasets/gregorut/videogamesales

In [77]:
df = pd.read_excel("videogame_sales.xlsx")
df.head()

,Rank,Name,Franchise,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,NaN,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,Mario,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Mario,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,NaN,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,Pokemon,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


## Información adicional de la API de RAWG.io

De parte de la base de datos obtenida utilizando la API de **RAWG.io** se agregarán las siguientes columnas para complementar la base de datos descrita en el punto anterior:

- playtime: horas de juego promedio que ofrece el videojuego
- metacritic: puntaje agregado del sitio metacritic que corresponde al *critics score*, es decir, puntaje de criticos especializados
- rating: puntaje agregado del sitio RAWG.io, es decir, que corresponde a usuarios de los videojuegos
- exceptional: numero de valoraciones con clasificacion *exceptional*
- recommended: numero de valoraciones con clasificacion *recommended*
- meh: numero de valoraciones con clasificacion *meh*
- skip: numero de valoraciones con clasificación *skip*
- esrb_rating: clasificación de rango etario recomendado para el videojuego
- rawg_name: nombre del videojuego almacenado en RAWG.io para comprobar que el videojuego es el correcto

La documnentación de la API de RAWG se puede encontrar en https://rawg.io/apidocs

### Añadir columnas a descargar desde la API

In [78]:
# Se añaden las columnas a descargar con valores nulos
df["playtime"] = pd.NA
df["metacritic"] = pd.NA
df["rating"] = pd.NA
df["exceptional"] = pd.NA
df["recommended"] = pd.NA
df["meh"] = pd.NA
df["skip"] = pd.NA
df["esrb_rating"] = pd.NA
df["rawg_name"] = pd.NA

### Funcion para obtener los datos del videojuego buscados por nombre

In [79]:
async def get_game_data_by_name(session: ClientSession, name: str):
    """Funcion asincrónica que solicita a la API los datos buscados por nombre (name) y
    y los retorna en formato JSON."""

    params = {
        "key": API_KEY,
        "search": name,
        "page_size": 1
    }
    try:
        async with session.get(URL, params=params) as response:
            return await response.json()
    except:
        print(name)
        return

### Función para buscar todos los datos por nombres entregados por lista

In [80]:
async def get_all_data_by_name(names: list[str]):
    """Función asincrónica que genera una lista de requests asincrónicos para solicitar
    información a la API y garantiza que el orden de los resultados sea el mismo orden
    de la lista de nombres (names)"""
    
    async with aiohttp.ClientSession() as session:
        # Se utiliza un diccionario para garantizar el orden
        # final de los resultados
        tasks = {i: get_game_data_by_name(session, name)
                 for i, name in enumerate(names)}
        results = await asyncio.gather(*tasks.values())

        ordered_results = [results[i] for i in range(len(names))]
        return ordered_results

### Función principal de ejecución

In [81]:
def main(names: list[str]):
    """Punto de entrada de ejecución de los requests de la API"""
    return asyncio.run(get_all_data_by_name(names))

### Descargar datos desde la API

In [82]:
# Quitar comentario sólo cuando desee descargar
# names = df.Name
# results = main(names)

### Agregar datos descargados de la API al Dataframe

In [115]:
def add_results_to_df(df: pd.DataFrame, results, names:pd.Series):
    """Función que agrega al Dataframe df los resultados obtenidos de la API (results) y que
    siguen el orden de la serie de nombres (names)"""
    
    for i, x in enumerate(results):
        # Si no hay datos continuar
        if not x:
            continue
        
        # Se obtienen los resultados que se encuentran bajo la key=results
        # en el indice 0 de la lista (de acuerdo a la documentación de la API)
        print(i)
        if len(x["results"]) == 0:
            continue
        
        result = x["results"][0]

        esrb_rating = result["esrb_rating"]
        if esrb_rating:
            df.loc[df.Name == names.iloc[i], "esrb_rating"] = esrb_rating["name"]

        df.loc[df.Name == names.iloc[i], "playtime"] = result["playtime"]
        df.loc[df.Name == names.iloc[i], "metacritic"] = result["metacritic"]
        df.loc[df.Name == names.iloc[i], "rating"] = result["rating"]

        ratings = result["ratings"]
        for rating in ratings:
            df.loc[df.Name == names.iloc[i], rating["title"]] = rating["count"]

        df.loc[df.Name == names.iloc[i], "rawg_name"] = result["name"]

In [84]:
# add_results_to_df(df, results)

In [85]:
# df.info()

In [86]:
# df.to_csv("videogame_data.csv")

## Segundo intento

Se realiza un segundo intento de descarga ya que hay varios elementos que fallaron por desconexión, o tiempo de espera excedido.

In [87]:
# Cargar los datos del proceso anterior
df2 = pd.read_csv("videogame_data.csv")

In [88]:
# Se verifica la cantidad de datos nulos previos al segundo intento
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    16598 non-null  int64  
 1   Rank          16598 non-null  int64  
 2   Name          16598 non-null  object 
 3   Franchise     7993 non-null   object 
 4   Platform      16598 non-null  object 
 5   Year          16327 non-null  float64
 6   Genre         16598 non-null  object 
 7   Publisher     16540 non-null  object 
 8   NA_Sales      16598 non-null  float64
 9   EU_Sales      16598 non-null  float64
 10  JP_Sales      16598 non-null  float64
 11  Other_Sales   16598 non-null  float64
 12  Global_Sales  16598 non-null  float64
 13  playtime      8320 non-null   float64
 14  metacritic    3828 non-null   float64
 15  rating        8320 non-null   float64
 16  exceptional   5659 non-null   float64
 17  recommended   6611 non-null   float64
 18  meh           6256 non-nul

In [89]:
# Se crea la lista de nombres que fallaron en descargar
failed_to_download = df2[df2["playtime"].isna()].Name

In [90]:
# Se calcula la cantidad de videojuegos que fallaron en la descarga de datos
len(failed_to_download)

8278

In [98]:
# Se divide el proceso en varias listas para evitar que un error
# haga fallar toda la descarga (tenemos cantidad limitada de consultas
# a la API)
names21 = failed_to_download[0:1000]
names22 = failed_to_download[1000:2000]
names23 = failed_to_download[2000:3000]
names24 = failed_to_download[3000:4000]
names25 = failed_to_download[5000:6000]
names26 = failed_to_download[6000:7000]
names27 = failed_to_download[7000:8000]
names28 = failed_to_download[8000:8278]

In [138]:
# Se ejecuta la consulta a la API por etapas
# results21 = main(names21)
# results22 = main(names22)
# results23 = main(names23)
# results24 = main(names24)
# results25 = main(names25)
# results26 = main(names26)
# results27 = main(names27)
results28 = main(names28)

In [139]:
# Se verifica la cantidad de elementos no nulos (es decir, exitosos)
filtered28 = [x for x in results28 if x is not None]
len(filtered28)

278

In [140]:
# Se añaden los datos al Dataframe por etapas
# add_results_to_df(df2, results21, names21)
# add_results_to_df(df2, results22, names22)
# add_results_to_df(df2, results23, names23)
# add_results_to_df(df2, results24, names24)
# add_results_to_df(df2, results25, names25)
# add_results_to_df(df2, results26, names26)
# add_results_to_df(df2, results27, names27)
add_results_to_df(df2, results28, names28)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [141]:
# Se verifican si los datos han sido añadidos al Dataframe
df2[df2["Name"].isin(names28)]

,Unnamed: 0,Rank,Name,Franchise,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,...,Global_Sales,playtime,metacritic,rating,exceptional,recommended,meh,skip,esrb_rating,rawg_name
6753,6753,6755,SBK Superbike World Championship,SBKSuperbike,PS3,2008.0,Racing,Black Bean Games,0.12,0.11,...,0.25,0.0,NaN,0.00,NaN,1.0,NaN,NaN,NaN,SBK 08: Superbike World Championship
6858,6858,6860,Monster Jam,MonsterJam,PS2,2007.0,Racing,Activision,0.12,0.09,...,0.24,4.0,NaN,3.00,NaN,4.0,4.0,2.0,NaN,Monster Jam
7109,7109,7111,Ratchet & Clank: Full Frontal Assault,Ratchet&Clank,PS3,2012.0,Adventure,Sony Computer Entertainment,0.19,0.01,...,0.23,0.0,64.0,3.49,6.0,20.0,19.0,4.0,Everyone 10+,Ratchet & Clank: Full Frontal Assault
7420,7420,7422,SBK Superbike World Championship,SBKSuperbike,X360,2008.0,Racing,Black Bean Games,0.06,0.14,...,0.21,0.0,NaN,0.00,NaN,1.0,NaN,NaN,NaN,SBK 08: Superbike World Championship
7598,7598,7600,Monster Jam,MonsterJam,X360,2007.0,Racing,Activision,0.18,0.00,...,0.20,4.0,NaN,3.00,NaN,4.0,4.0,2.0,NaN,Monster Jam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16592,16592,16595,Plushees,NaN,DS,2008.0,Simulation,Destineer,0.01,0.00,...,0.01,0.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,Plushees
16593,16593,16596,Woody Woodpecker in Crazy Castle 5,NaN,GBA,2002.0,Platform,Kemco,0.01,0.00,...,0.01,0.0,NaN,0.00,NaN,NaN,1.0,NaN,NaN,Woody Woodpecker (2001)
16595,16595,16598,SCORE International Baja 1000: The Official Game,NaN,PS2,2008.0,Racing,Activision,0.00,0.00,...,0.01,0.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,Score International Baja 1000
16596,16596,16599,Know How 2,NaN,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,...,0.01,0.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,Paper Know-how


### Guardar versión final de la base de datos

In [143]:
df2.to_csv("videogame_data_final.csv")

### Cargar base de datos para verificación final

In [144]:
df3 = pd.read_csv("videogame_data_final.csv")

In [145]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  16598 non-null  int64  
 1   Unnamed: 0    16598 non-null  int64  
 2   Rank          16598 non-null  int64  
 3   Name          16598 non-null  object 
 4   Franchise     7993 non-null   object 
 5   Platform      16598 non-null  object 
 6   Year          16327 non-null  float64
 7   Genre         16598 non-null  object 
 8   Publisher     16540 non-null  object 
 9   NA_Sales      16598 non-null  float64
 10  EU_Sales      16598 non-null  float64
 11  JP_Sales      16598 non-null  float64
 12  Other_Sales   16598 non-null  float64
 13  Global_Sales  16598 non-null  float64
 14  playtime      15869 non-null  float64
 15  metacritic    5622 non-null   float64
 16  rating        15869 non-null  float64
 17  exceptional   8524 non-null   float64
 18  recommended   10458 non-nu